<a href="https://colab.research.google.com/github/aschoudry/ML_imageClassifier/blob/main/pycbc_generate_cwt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycbc
import numpy as np
import pycbc
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
%matplotlib inline
from pycbc.waveform import td_approximants, fd_approximants, get_td_waveform
from pycbc.detector import Detector
import cv2
from scipy.interpolate import interp1d
import pywt


     |████████████████████████████████| 6.7 MB 12.6 MB/s 
     |████████████████████████████████| 52 kB 786 kB/s 
     |████████████████████████████████| 75 kB 4.6 MB/s 
     |████████████████████████████████| 1.6 MB 62.6 MB/s 
     |████████████████████████████████| 27.6 MB 88 kB/s 
     |████████████████████████████████| 1.0 MB 57.7 MB/s 
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 3.5 MB 60.9 MB/s 
  Created wheel for lscsoft-glue: filename=lscsoft_glue-2.0.0-cp37-cp37m-linux_x86_64.whl size=416867 sha256=f0e668156ff0c7abbb0edb495ad25e951222fd0459ae116abf01d0d7f126967d
  Stored in directory: /root/.cache/pip/wheels/9d/ed/9e/1f9cad5e15359dc17954b893dd486760b5cb1ffd97a8405d1e
  Created wheel for mpld3: filename=mpld3-0.5.5-py3-none-any.whl size=200907 sha256=0bc84762755e078655eec53bd27a5ab398558d689d18ed061d64ad758af3241c
  Stored in directory: /root/.cache/pip/wheels/3b/f3/08/a11447ac4ea31d7603142e76b4cba9e48b2fa7d82b76b69731
  Crea

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
from PIL import Image

In [ ]:
np.random.seed(1)

#Define the Detectors
det_h1 = Detector('H1')
det_l1 = Detector('L1')
det_v1 = Detector('V1')

m_vec_train = np.random.uniform(10, 100, 1500)


for n in m_vec_train:
    #Define the GW params
    gwapprox = 'SEOBNRv4'
    #print(gwapprox)
    hp, hc = get_td_waveform(approximant=gwapprox,
                             mass1=n,
                             mass2=n,
                             delta_t=1.0/4096,
                             spin1z=0.0,
                             spin2z=0.0,
                             inclination= 2 * np.pi,
                             coa_phase= 2 * np.pi,
                             phase_order = 0,
                             f_lower=20.0,
                             distance=100,
                            )

    
    # Choose a GPS end time, sky location, and polarization phase for the merger
    # NOTE: Right ascension and polarization phase runs from 0 to 2pi
    #       Declination runs from pi/2. to -pi/2 with the poles at pi/2. and -pi/2.
    end_time = 1192529720 #+ np.random.randint(1192529720//100000)
    declination = 0*np.pi * np.random.rand() - np.pi/2
    right_ascension = 2 * np.pi #* np.random.rand()
    polarization = 2 * np.pi # * np.random.rand()
    hp.start_time += end_time
    hc.start_time += end_time

    signal_h1 = det_h1.project_wave(hp, hc,  right_ascension, declination, polarization)
    signal_l1 = det_l1.project_wave(hp, hc,  right_ascension, declination, polarization)
    signal_v1 = det_v1.project_wave(hp, hc,  right_ascension, declination, polarization)    
    minlen = np.min( [len(signal_h1), len(signal_l1), len(signal_v1)] )
    data = np.stack( (signal_h1[:minlen], signal_l1[:minlen], signal_v1[:minlen]),  ) * 1e19
    #print(data.shape)
    
    if data.shape[1]>4096:
        data = data[:,data.shape[1]-4096:]
        for N in range(80):
            data[:,N] *= 1./(N+1)
    
    if len(hp)<4096:
        for N in range(80):
            data[:,N] *= 1./(N+1)
        data = np.pad(data, ((0,0),(4096-data.shape[1],0)) )

    cwt, freqs = pywt.cwt(data, scales=np.arange(1, 95, 0.62), wavelet='cmor1.5-0.95', sampling_period=1/2048, method='fft')
    cwt = cwt.transpose(0,2,1)
  
    cwt = np.log1p( np.abs(cwt) )
    
    cwt = cv2.resize(cwt,(256, 256)) 
    cwt -= cwt.min()
    cwt /= cwt.max()
    cwt=cwt*255
    cwt = cwt.astype(np.uint8)
    
    im = Image.fromarray(cwt, 'RGB')
    im.save('/content/gdrive/MyDrive/pycbc_image_data/train_m_'+str(n)+'_.png')
    
    

In [ ]:
m_vec_test = np.random.uniform(10, 100, 200)

for n in m_vec_test:
    #Define the GW params
    gwapprox = 'SEOBNRv4'
    #print(gwapprox)
    hp, hc = get_td_waveform(approximant=gwapprox,
                             mass1=n,
                             mass2=n,
                             delta_t=1.0/4096,
                             spin1z=0.0,
                             spin2z=0.0,
                             inclination= 2 * np.pi,
                             coa_phase= 2 * np.pi,
                             phase_order = 0,
                             f_lower=20.0,
                             distance=100,
                            )

    
    # Choose a GPS end time, sky location, and polarization phase for the merger
    # NOTE: Right ascension and polarization phase runs from 0 to 2pi
    #       Declination runs from pi/2. to -pi/2 with the poles at pi/2. and -pi/2.
    end_time = 1192529720 #+ np.random.randint(1192529720//100000)
    declination = 0*np.pi * np.random.rand() - np.pi/2
    right_ascension = 2 * np.pi #* np.random.rand()
    polarization = 2 * np.pi # * np.random.rand()
    hp.start_time += end_time
    hc.start_time += end_time

    signal_h1 = det_h1.project_wave(hp, hc,  right_ascension, declination, polarization)
    signal_l1 = det_l1.project_wave(hp, hc,  right_ascension, declination, polarization)
    signal_v1 = det_v1.project_wave(hp, hc,  right_ascension, declination, polarization)    
    minlen = np.min( [len(signal_h1), len(signal_l1), len(signal_v1)] )
    data = np.stack( (signal_h1[:minlen], signal_l1[:minlen], signal_v1[:minlen]),  ) * 1e19
    #print(data.shape)
    
    if data.shape[1]>4096:
        data = data[:,data.shape[1]-4096:]
        for N in range(80):
            data[:,N] *= 1./(N+1)
    
    if len(hp)<4096:
        for N in range(80):
            data[:,N] *= 1./(N+1)
        data = np.pad(data, ((0,0),(4096-data.shape[1],0)) )

    cwt, freqs = pywt.cwt(data, scales=np.arange(1, 95, 0.62), wavelet='cmor1.5-0.95', sampling_period=1/2048, method='fft')
    cwt = cwt.transpose(0,2,1)
  
    cwt = np.log1p( np.abs(cwt) )
    
    cwt = cv2.resize(cwt,(256, 256)) 
    cwt -= cwt.min()
    cwt /= cwt.max()
    cwt=cwt*255
    cwt = cwt.astype(np.uint8)
    
    im = Image.fromarray(cwt, 'RGB')
    im.save('/content/gdrive/MyDrive/pycbc_image_data/test_m_'+str(n)+'_.png')